## Setup
1. Run YAP API server
1. `pip install -r requirements.txt`
1. Run NEMO API server `uvicorn api_main:app --port 8090`
1. Have a look at the swagger OpenAPI documentation by opening http://localhost:8090/docs in your browser

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
import pandas as pd
import numpy as np
from time import sleep

In [ ]:
data = pd.read_csv("/home/esther/PycharmProjects/ANLP_analogies/Data/Datasets/AnalogiesData.csv")
sentences = data[['base_description','option_1_desc','option_2_desc','option_3_desc','option_4_desc']]
# Concatenate all cells in each column
texts = sentences.values.flatten().tolist()
texts = [x for x in texts if type(x)==str]

file = open('sentences.txt','w')
for s in texts:
    if type(s)==str:
        file.write(s+"\n")
file.close()

In [ ]:
import requests
import json


def run_yap(text):
    localhost_yap = "http://localhost:8000/yap/heb/joint"
    data = json.dumps({'text': "{}  ".format(text)})  # input string ends with two space characters
    headers = {'content-type': 'application/json'}
    response = requests.get(url=localhost_yap, data=data, headers=headers)
    json_response = response.json()
    keys = list(json_response.keys())
    return json_response.get(keys[1])


def df_from_yap_output(output, text, res, is_word):
    add_data = pd.DataFrame(columns = ["sentence", "word", "POS"])
    for row in output.split('\n'):
        morph_list = row.split('\t')
        if len(morph_list) < 6:
            continue
        assert morph_list[4] == morph_list[5]
        if is_word:
            new_row = {"sentence": "", "word": morph_list[2], "POS": morph_list[5]}
        else:
            new_row = {"sentence": text, "word": morph_list[2], "POS": morph_list[5]}
        new_row = pd.DataFrame(new_row, index=[0])
        add_data = pd.concat([add_data, new_row], ignore_index=True)
    return add_data

In [ ]:
df1 = pd.read_csv("yap_morph_1_outof_5.csv", index_col=0)
df2 = pd.read_csv("yap_morph_2_outof_6.csv", index_col=0)
df3 = pd.read_csv("yap_morph_3_outof_9.csv", index_col=0)
df4 = pd.read_csv("yap_morph_4_outof_10.csv", index_col=0)
df5 = pd.read_csv("yap_morph_5_outof_10.csv", index_col=0)
df6 = pd.read_csv("yap_morph_6_outof_10.csv", index_col=0)
yap_morph = pd.concat([df1, df2, df3, df4, df5, df6])
yap_morph.to_csv("yap_morph.csv")